<a href="https://colab.research.google.com/github/VMLmartech/encoderinstagram/blob/main/INSTAGRAM_ENCODER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install gspread
!pip install ftfy
!pip install gspread dash pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dash import Dash, dcc, html, Input, Output, dash_table

In [17]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

# Leer datos desde Google Sheets
gc = gspread.authorize(creds)
worksheet = gc.open('JUMBO-BBDD').worksheet('IG_COMMENTS')
rows = worksheet.get_all_values()

# Convertir a DataFrame
df = pd.DataFrame(rows[1:], columns=rows[0])  # Asumiendo que la primera fila son encabezados

# Aplicar fix_text para corregir caracteres mal codificados
df = df.applymap(fix_text)

# Ver resultado
print(df.head())



<ipython-input-17-9e132f49f66f>:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(fix_text)


       User Id                      Username         Comment Id  \
0  51223151737                     kailychi1  17932657652601213   
1   4110503699  rocio_jazmin_alarcon_herrera  18099390685324786   
2   1754003455     hablandoconelpropiojanusz  17985996749136753   
3    212201580              chocolates_jumbo  18290008579116821   
4    212201580              chocolates_jumbo  18023586430510352   

                                      Comment  \
0                                ❤️❤️❤️❤️❤️❤️   
1                     Amo la Flow blanca ü§§   
2  ü§§ü§§ü§§ü§§ Saludos @chocolates_jumbo   
3           @rocio_jazmin_alarcon_herrera 😍😍😍   
4    @hablandoconelpropiojanusz ¡Saludos! ✌️😎   

                                         Profile URL  \
0                https://www.instagram.com/kailychi1   
1  https://www.instagram.com/rocio_jazmin_alarcon...   
2  https://www.instagram.com/hablandoconelpropioj...   
3         https://www.instagram.com/chocolates_jumbo   
4         https://www.

In [18]:
from ftfy import fix_text

def fix_encoding(text):
    try:
        # Try decoding with utf-8 first
        return text.encode('utf-8').decode('utf-8')
    except UnicodeEncodeError:
        # If utf-8 fails, try latin-1 and then fix with ftfy
        return fix_text(text.encode('latin1').decode('latin1', errors='ignore'))
    except AttributeError: # If the value is already decoded ignore it:
        return text


df["Comment"] = df["Comment"].apply(fix_encoding)
df["Comment"] = df["Comment"].apply(fix_text)

In [19]:
# prompt: append sheet "Comment" column

# Assuming 'df' is already defined from the previous code

# Append a new column named "New_Comment" to the DataFrame
df["New_Comment"] = ""  # Initialize the column with empty strings


# Update the worksheet with the new column
worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '11M9xHDCtFW627UVEAEygpuUk7RBXOSm9H-e-YMp17sA',
 'updatedRange': 'IG_COMMENTS!A1:I1106',
 'updatedRows': 1106,
 'updatedColumns': 9,
 'updatedCells': 9954}

In [20]:
# Asegurar que las columnas correctas tienen texto
df["Comment"] = df["Comment"].astype(str)
df["Url_Post"] = df["Url_Post"].astype(str)

In [21]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Comentarios de Instagram - Filtro Dinámico"),

    # Dropdown para seleccionar la URL del post
    dcc.Dropdown(
        id="url_post_filter",
        options=[{"label": url, "value": url} for url in df["Url_Post"].unique()],
        placeholder="Selecciona una URL del post",
        multi=False
    ),

    # Input para buscar palabras en los comentarios
    dcc.Input(
        id="search_term",
        type="text",
        placeholder="Buscar palabras en comentarios",
        debounce=True
    ),

    # Tabla de comentarios
    dash_table.DataTable(
        id="comments_table",
        columns=[{"name": col, "id": col} for col in df.columns],
        page_size=10,
        style_table={'overflowX': 'auto'}
    )
])

# Callback para filtrar la tabla dinámicamente
@app.callback(
    Output("comments_table", "data"),
    Input("url_post_filter", "value"),
    Input("search_term", "value")
)
def update_table(selected_url, search_term):
    filtered_df = df.copy()

    # Filtrar por URL del post
    if selected_url:
        filtered_df = filtered_df[filtered_df["Url_Post"] == selected_url]

    # Filtrar por palabras clave en los comentarios
    if search_term:
        filtered_df = filtered_df[filtered_df["Comment"].str.contains(search_term, case=False, na=False)]

    return filtered_df.to_dict("records")

if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>